In [1]:
import pandas as pd
import numpy as np
import os
import importlib
from datetime import datetime,date
import glob

In [2]:
# parentDir = os.path.abspath(os.path.join(os.getcwd(),'..'))
# os.chdir(parentDir)

# Full Run

# Check data_loader

## Check load_transaction

In [3]:


def load_transaction(data_folder_path, transaction_file_pattern):
    transaction_files = _gather_transaction_files(data_folder_path,transaction_file_pattern)
    transactions = _combine_transaction_files(transaction_files)
    transactions = _clean_transactions(transactions)
    print(f"The latest transaction date is {transactions['Run Date'].max()}")
    return transactions


def _gather_transaction_files(data_folder_path,transaction_file_pattern):
    transaction_file_path_pattern = os.path.join(
        data_folder_path, transaction_file_pattern
    )
    transaction_files = glob.glob(transaction_file_path_pattern)
    return transaction_files

def _combine_transaction_files(transaction_files):
    transaction_list = [
        pd.read_csv(file, usecols=range(14)) for file in transaction_files
    ]
    transactions = pd.concat(transaction_list, ignore_index=True)
    return transactions

def _remove_NA_value(df,colName):
    df_copy = df.copy()
    df_copy = df_copy[
        df_copy[colName].notna()
    ] 
    return df_copy

def _remove_leading_space(df,colName):
    df_copy = df.copy()
    df_copy[colName] = df_copy[colName].str.lstrip()
    return df_copy

def _str_to_date(df, colName, format):
    df_copy = df.copy()
    df_copy[colName] = pd.to_datetime(
        df_copy[colName], format=format
    ).dt.date
    return df_copy

def _add_Transfer_symbol(df):
    df_copy = df.copy()
    df_copy.loc[df_copy["Symbol"] == "  ", "Symbol"] = "Transfer"
    return df_copy

def _sort_df_by_column(df, colName):
    df_copy = df.copy()
    df_copy = df_copy.sort_values(by=colName).reset_index(
        drop=True
    )
    return df_copy


def _clean_transactions(transactions):
    transactions = _remove_NA_value(transactions,"Amount ($)")
    transactions = _remove_leading_space(transactions,"Run Date")
    transactions = _str_to_date(transactions,"Run Date","%m/%d/%Y")
    transactions = _str_to_date(transactions,"Settlement Date","%m/%d/%Y")
    transactions = _add_Transfer_symbol(transactions)
    transactions = _remove_leading_space(transactions,"Symbol")
    transactions = _sort_df_by_column(transactions,"Run Date")
    return transactions


In [4]:
data_folder_path = "./data"
transaction_file_pattern = "Accounts_History_*.csv"
position_file_pattern = "Portfolio_Positions_*.csv"

In [5]:
transaction_files = _gather_transaction_files(data_folder_path,transaction_file_pattern)
transactions = _combine_transaction_files(transaction_files)
transactions = _clean_transactions(transactions)

In [6]:
# transactions = _remove_NA_value(transactions,"Amount ($)")
# transactions = _remove_leading_space(transactions,"Run Date")
# transactions = _str_to_date(transactions,"Run Date","%m/%d/%Y")
# transactions = _str_to_date(transactions,"Settlement Date","%m/%d/%Y")
# transactions = _add_Transfer_symbol(transactions)
# transactions = _remove_leading_space(transactions,"Symbol")
# transactions = _sort_df_by_column(transactions,"Run Date")

## Check load_position

In [7]:

def load_position(data_folder_path, position_file_pattern):
    position_file = _gather_position_files(data_folder_path, position_file_pattern)

    if not position_file is None:
        position = pd.read_csv(position_file)
        position = _clean_position(position)
    else:
        print("No position file found.")
    return position

def _gather_position_files(data_folder_path, position_file_pattern):
    position_file_path_pattern = os.path.join(data_folder_path, position_file_pattern)
    position_files = glob.glob(position_file_path_pattern)
    position_file = _find_latest_position_file(position_files)
    return position_file


def _find_latest_position_file(position_files):
    latest_file = None
    latest_date = None

    for file_path in position_files:
        file_name = os.path.basename(file_path)
        date_str = file_name.split("_")[-1].replace(".csv", "")
        file_date = datetime.strptime(date_str, "%b-%d-%Y")

        if latest_date is None or file_date > latest_date:
            latest_date = file_date
            latest_file = file_path

    return latest_file


def _clean_position(position):
    position = _remove_NA_value(position,"Current Value")
    position = _transfer_dollar_to_float(position, "Current Value")
    position = _transfer_dollar_to_float(position, "Cost Basis Total")
    return position


def _transfer_dollar_to_float(df, colNames):
    """
    Change "$123,456" to 123456.0, and "--" to 0.0
    """
    df_copy = df.copy()
    # Replace any "--" with "$0"
    cleaned = df_copy[colNames].str.replace("--", "$0", regex=False)
    # Remove dollar sign and commas, then convert to float
    cleaned = cleaned.str.replace("$", "", regex=False).str.replace(",", "", regex=False)
    df_copy[colNames] = cleaned.astype(float)
    return df_copy


In [8]:
data_folder_path = "./data"
transaction_file_pattern = "Accounts_History_*.csv"
position_file_pattern = "Portfolio_Positions_*.csv"

In [9]:
position_file = _gather_position_files(data_folder_path, position_file_pattern)
position_file

'./data/Portfolio_Positions_Aug-05-2025.csv'

In [10]:
position = pd.read_csv(position_file)
position = _clean_position(position)

In [11]:
# position = _remove_NA_value(position,"Current Value")
# position = _transfer_dollar_to_float(position, "Current Value")
# position = _transfer_dollar_to_float(position, "Cost Basis Total")

# Check Portfolio

In [ ]:
from scipy.optimize import newton
import numpy as np
def compute_irr(cashflows, dates, cob):
    """用 Newton-Raphson 方法计算 IRR（连续复利）"""
    def npv(r):
        return sum(cf * np.exp(-r * (cob - d).days / 365.0) for cf, d in zip(cashflows, dates))
    try:
        result = -newton(npv, 0.1)
    except RuntimeError:
        result = np.nan
    return result

def display_percentage(df, colNames: list):
    df_copy = df.copy()
    for col in colNames:
        df_copy[col] = df_copy[col].apply(lambda x: f"{x:.2%}")
    return df_copy

In [ ]:
class Portfolio:
    def __init__(self, transactions, position):
        self.transactions = transactions
        self.position = position
        self.cob = date.today()
        
        account_number_dic = {
            "Individual":'Z23390746',
            "401k":'86964',
            "HSA":'241802439',
            "Cash":'Z06872898'
        }
        
        self.individualTransactions = transactions[transactions['Account Number']==account_number_dic["Individual"]]
        self.individualPosition = position[position['Account Number']==account_number_dic["Individual"]]
        self.pensionTransactions = transactions[transactions['Account Number']==account_number_dic["401k"]]
        self.pensionPosition = position[position['Account Number']==account_number_dic["401k"]]
        self.HSATransactions = transactions[transactions['Account Number']==account_number_dic["HSA"]]
        self.HSAPosition = position[position['Account Number']==account_number_dic["HSA"]]
        self.cashTransactions = transactions[transactions['Account Number']==account_number_dic["Cash"]]
        self.cashPosition = position[position['Account Number']==account_number_dic["Cash"]]
        
        
        self.cashSymbols = ['FZFXX**','FZFXX']
        self.otherSymbols = ['Pending Activity']
        self.contributionSymbols = ['','Transfer']
        self.bondSymbols = self.get_bond_symbol_list()
        self.stockSymbols = self.get_stock_symbol_list()
        
    def get_individual_account_summary(self):
        bondSymbols = self.get_bond_symbol_list()
        bondTotalValue = self.get_total_symbols_value(bondSymbols)
        bondTotalIrr = self.get_combined_symbol_irr(bondSymbols)
        bondHoldingPeriod = self.get_combined_symbol_holding_period(bondSymbols)
        
        stockSymbols = self.get_stock_symbol_list()
        stockTotalValue = self.get_total_symbols_value(stockSymbols)
        stockTotalIrr = self.get_combined_symbol_irr(stockSymbols)
        stockHoldingPeriod = self.get_combined_symbol_holding_period(stockSymbols)
        
        cashSymbols = self.cashSymbols
        cashTotalValue = self.get_total_symbols_value(cashSymbols)
        cashTotalIrr = self.get_combined_symbol_irr(cashSymbols)
        cashHoldingPeriod = self.get_combined_symbol_holding_period(cashSymbols)
        
        totalIRR = self.get_total_irr()
        
        totalValue = self.individualPosition['Current Value'].sum()
        result = pd.DataFrame({
            'Type':['bond','stock','cash','Total'],
            'Value': [bondTotalValue,stockTotalValue,cashTotalValue,totalValue],
            'Percentage':[bondTotalValue,stockTotalValue,cashTotalValue,totalValue]/totalValue,
            'IRR':[bondTotalIrr, stockTotalIrr, cashTotalIrr,totalIRR]
        } 
        )
        return result
    
        
    def get_all_stock_summary(self):
        stockSymbols = self.get_stock_symbol_list()
        currentValueResult = self.get_symbol_current_values(stockSymbols)
        irrResult = self.get_symbol_irrs(stockSymbols)
        holdingPeriodResult = self.get_symbol_holding_period(stockSymbols)
        result = pd.merge(currentValueResult, irrResult, on='Symbol')
        result = pd.merge(result, holdingPeriodResult, on='Symbol')
        result = result.sort_values(by='Current Value', ascending=False)
        return result
    
    def get_all_bond_summary(self):
        bondSymbols = self.get_bond_symbol_list()
        currentValueResult = self.get_symbol_current_values(bondSymbols)
        irrResult = self.get_symbol_irrs(bondSymbols)
        holdingPeriodResult = self.get_symbol_holding_period(bondSymbols)
        result = pd.merge(currentValueResult, irrResult, on='Symbol')
        result = pd.merge(result, holdingPeriodResult, on='Symbol')
        result = result.sort_values(by='Current Value', ascending=False)
        return result
    
    def get_combined_symbol_holding_period(self, listSymbols: list, unit = 30):
        subTransactions = self.transactions[self.transactions['Symbol'].isin(listSymbols)]
        buyTranactions = subTransactions[subTransactions['Amount ($)']<0]
        df = buyTranactions.copy()
        df['Days Held'] = (self.cob - df['Run Date']).apply(lambda x: x.days)
        df['Weight'] = df['Amount ($)'].abs()
        totalWeightedHold = (df['Days Held'] * df['Weight']).sum() / df['Weight'].sum()/ unit
        return totalWeightedHold
        
    def get_symbol_holding_period(self, listSymbols: list, unit = 30):
        subTransactions = self.transactions[self.transactions['Symbol'].isin(listSymbols)]
        buyTranactions = subTransactions[subTransactions['Amount ($)']<0]
        df = buyTranactions.copy()
        df['Days Held'] = (self.cob - df['Run Date']).apply(lambda x: x.days)
        df['Weight'] = df['Amount ($)'].abs()
        totalWeightedHold = (df['Days Held'] * df['Weight']).sum() / df['Weight'].sum()/ unit
        totalWeightedHoldRow = pd.DataFrame({'Symbol': ['Total'], 'Weighted Avg Holding Period': [totalWeightedHold]})

        # 分组计算加权平均
        weightedHold = (
            df.groupby('Symbol')
            .apply(lambda g: (g['Days Held'] * g['Weight']).sum() / g['Weight'].sum()/ unit, include_groups=False)
            .reset_index(name='Weighted Avg Holding Period')
        )
        
        weightedHold = pd.concat([weightedHold, totalWeightedHoldRow], ignore_index=True)
        return weightedHold
    
    
    def get_symbol_current_values(self, listSymbols: list):
        resultList = []
        totalCurrentValue = self.get_total_symbols_value(listSymbols)
        for symbol in listSymbols:
            currentValue = self.get_symbol_current_value(symbol)
            currentValuePercent = currentValue/totalCurrentValue
            resultList.append({
                'Symbol': symbol,
                'Current Value': currentValue,
                'Percentage': currentValuePercent
            })
        resultList.append({
                'Symbol': 'Total',
                'Current Value': totalCurrentValue,
                'Percentage': 1
            })
        return pd.DataFrame(resultList)
    
    def get_symbol_current_value(self,symbol):
        try:
            value = self.position.loc[self.position['Symbol']==symbol, 'Current Value'].values[0]
        except:
            value = 0
        return value
        
    def get_total_symbols_value(self, symbols: list):
        Position = self.individualPosition[self.individualPosition['Symbol'].isin(symbols)]
        totalValue = Position['Current Value'].sum()
        return totalValue
    
    def get_symbol_irrs(self, listSymbols: list):
        resultList = []
        for symbol in listSymbols:
            irr = self.get_combined_symbol_irr([symbol])
            resultList.append({
                'Symbol': symbol,
                'IRR': irr
            })
        totalIrr = self.get_combined_symbol_irr(listSymbols)
        resultList.append({
                'Symbol': 'Total',
                'IRR': totalIrr
            })
        return pd.DataFrame(resultList)
    
    
    def get_total_irr(self):
        trans = self.individualTransactions[self.individualTransactions['Symbol'].isin(self.contributionSymbols)]
        cashflows = trans['Amount ($)'].tolist()
        cashflows = [-x for x in cashflows]
        dates = trans['Run Date'].tolist()
        current_value = self.individualPosition['Current Value'].sum()
        cashflows.append(current_value)
        dates.append(self.cob)
        irr = compute_irr(cashflows, dates, self.cob)
        return irr
    
    def get_combined_symbol_irr(self, listSymbols: list):
        trans = self.transactions[self.transactions['Symbol'].isin(listSymbols)]
        cashflows = trans['Amount ($)'].tolist()
        dates = trans['Run Date'].tolist()
        current_value = self.position.loc[self.position['Symbol'].isin(listSymbols), 'Current Value'].sum()
        cashflows.append(current_value)
        dates.append(self.cob)
        irr = compute_irr(cashflows, dates, self.cob)
        return irr
    
    def get_stock_symbol_list(self):
        symbols = self.individualTransactions['Symbol'].unique()
        stockSymbols = [
            sym for sym in symbols
            if sym not in self.cashSymbols
            and not sym.startswith('91')
            and sym not in self.otherSymbols
            and sym not in self.contributionSymbols
        ]
        return stockSymbols
    
    def get_bond_symbol_list(self):
        symbols = self.individualTransactions['Symbol'].unique()
        bondSymbols = [
            sym for sym in symbols
            if  sym.startswith('91')
        ]
        return bondSymbols
    

## Check get_individual_account_summary

In [45]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)
portfolio = Portfolio(transactions, position)

The latest transaction date is 2025-08-05


In [46]:
result = portfolio.get_individual_account_summary()
display_percentage(result,['Percentage','IRR'])

,Type,Value,Percentage,IRR
0,bond,746743.50,62.47%,4.38%
1,stock,423303.14,35.41%,16.32%
2,cash,25294.19,2.12%,nan%
3,Total,1195340.83,100.00%,8.70%


In [42]:
trans = portfolio.individualTransactions[portfolio.individualTransactions['Symbol'].isin(portfolio.contributionSymbols)]
cashflows = trans['Amount ($)'].tolist()
cashflows = [-x for x in cashflows]
sum(cashflows)

-1100000.0

In [43]:
dates = trans['Run Date'].tolist()
current_value = portfolio.individualPosition['Current Value'].sum()
cashflows.append(current_value)
dates.append(portfolio.cob)
irr = compute_irr(cashflows, dates, portfolio.cob)
irr

np.float64(0.08698187517784058)

In [27]:
cashflows

[1000.0,
 29000.0,
 30000.0,
 40000.0,
 1500.0,
 148500.0,
 -148500.0,
 148500.0,
 100000.0,
 -100000.0,
 100000.0,
 -100000.0,
 140000.0,
 50000.0,
 10000.0,
 50000.0,
 50000.0,
 50000.0,
 150000.0,
 np.float64(1195340.83)]

In [276]:
bondPosition = portfolio.individualPosition[portfolio.individualPosition['Symbol'].isin(portfolio.bondSymbols)]
bondPositionValue = bondPosition['Current Value'].sum()
stockPosition = portfolio.individualPosition[portfolio.individualPosition['Symbol'].isin(portfolio.stockSymbols)]
stockPositionValue = stockPosition['Current Value'].sum()
cashPosition = portfolio.individualPosition[portfolio.individualPosition['Symbol'].isin(portfolio.cashSymbols)]
cashPositionValue = cashPosition['Current Value'].sum()
[bondPositionValue,stockPositionValue,cashPositionValue]

[np.float64(746743.5), np.float64(423303.14), np.float64(25294.19)]

In [277]:
[bondPositionValue+stockPositionValue+cashPositionValue,portfolio.individualPosition['Current Value'].sum()]

[np.float64(1195340.83), np.float64(1195340.83)]

## Check get_all_stock_summary

In [278]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)

The latest transaction date is 2025-08-05


In [279]:
portfolio = Portfolio(transactions, position)

In [280]:
result = portfolio.get_all_stock_summary()
display_percentage(result,['Percentage','IRR'])

,Symbol,Current Value,Percentage,IRR,Weighted Avg Holding Years
21,Total,423303.14,100.00%,16.32%,0.997435
8,FXAIX,163865.08,38.71%,19.64%,0.779858
11,FSKAX,113381.14,26.78%,13.48%,0.924055
10,FSPSX,69497.82,16.42%,13.18%,1.102000
13,MSFT,15867.00,3.75%,17.02%,1.056837
0,AAPL,14233.47,3.36%,10.25%,2.873775
7,TSLA,6193.40,1.46%,32.69%,1.520608
9,MCD,6013.60,1.42%,14.91%,1.216643
14,BRKB,4661.80,1.10%,1.70%,0.786301
4,GOOGL,3888.10,0.92%,30.99%,2.452100


In [281]:
stockSymbols = portfolio.get_stock_symbol_list()
currentValueResult = portfolio.get_symbol_current_values(stockSymbols)
currentValueResult

,Symbol,Current Value,Percentage
0,AAPL,14233.47,0.033625
1,SBUX,2707.05,0.006395
2,JPM,2915.42,0.006887
3,AXP,2981.20,0.007043
4,GOOGL,3888.10,0.009185
5,AMZN,2136.90,0.005048
6,NKE,2230.05,0.005268
7,TSLA,6193.40,0.014631
8,FXAIX,163865.08,0.387110
9,MCD,6013.60,0.014206


In [282]:
listSymbols = stockSymbols
resultList = []
totalCurrentValue = portfolio.get_total_symbols_value(listSymbols)
for symbol in listSymbols:
    currentValue = portfolio.get_symbol_current_value(symbol)
    currentValuePercent = currentValue/totalCurrentValue
    resultList.append({
        'Symbol': symbol,
        'Current Value': currentValue,
        'Percentage': currentValuePercent
    })
resultList.append({
        'Symbol': 'Total',
        'Current Value': totalCurrentValue,
        'Percentage': 1
    })

In [283]:
symbol

'COST'

In [284]:
irrResult = portfolio.get_symbol_irrs(stockSymbols)
irrResult

,Symbol,IRR
0,AAPL,0.102480
1,SBUX,0.065428
2,JPM,0.323571
3,AXP,0.245922
4,GOOGL,0.309914
5,AMZN,0.330231
6,NKE,-0.071349
7,TSLA,0.326888
8,FXAIX,0.196376
9,MCD,0.149059


In [285]:
holdingPeriod = portfolio.get_symbol_holding_period(stockSymbols)
holdingPeriod

,Symbol,Weighted Avg Holding Years
0,AAPL,2.873775
1,AMZN,2.454795
2,AXP,2.928767
3,BRKB,0.786301
4,COKE,1.083672
5,COST,0.073973
6,FSKAX,0.924055
7,FSPSX,1.102000
8,FXAIX,0.779858
9,GOOGL,2.452100


In [286]:
df = portfolio.individualTransactions[portfolio.individualTransactions['Amount ($)']<0].copy()

In [287]:
df['Days Held'] = (portfolio.cob - df['Run Date']).apply(lambda x: x.days)
df['Weight'] = df['Amount ($)'].abs()

# 分组计算加权平均
weighted_hold = (
    df.groupby('Symbol')
      .apply(lambda g: (g['Days Held'] * g['Weight']).sum() / g['Weight'].sum()/ 365, include_groups=False)
      .reset_index(name='Weighted Avg Holding Years')
)
# weighted_hold['Weighted Avg Holding Years'] = weighted_hold['Weighted Avg Holding Years'] / 365
weighted_hold

,Symbol,Weighted Avg Holding Years
0,912796ZV4,0.728767
1,912797GB7,1.326027
2,912797GN1,1.591781
3,912797GW1,0.958904
4,912797GZ4,1.591781
...,...,...
72,PG,0.073973
73,SBUX,2.396871
74,TSLA,1.520608
75,Transfer,1.280525


## Check get_bond_irr

In [288]:
data_folder_path = './data'
transaction_file_pattern = 'Accounts_History_*.csv'
position_file_pattern = 'Portfolio_Positions_*.csv'
transactions = load_transaction(data_folder_path, transaction_file_pattern)
position = load_position(data_folder_path, position_file_pattern)

The latest transaction date is 2025-08-05


In [289]:
portfolio = Portfolio(transactions, position)

In [290]:
result = portfolio.get_all_bond_summary()
display_percentage(result,['Percentage','IRR'])

,Symbol,Current Value,Percentage,IRR,Weighted Avg Holding Years
54,Total,746743.5,100.00%,4.38%,0.499727
46,912797PW1,149338.5,20.00%,3.87%,0.064664
50,912797QU4,149253.0,19.99%,3.50%,0.071201
48,912797MG9,99976.0,13.39%,4.28%,0.115068
49,912797QL4,99751.0,13.36%,3.98%,0.109589
51,912797QT7,99586.0,13.34%,3.54%,0.071233
53,912797QW0,99336.0,13.30%,0.24%,0.013699
52,912797NA1,49503.0,6.63%,3.53%,0.021918
33,912797MM6,0.0,0.00%,4.12%,0.536986
36,912797PT8,0.0,0.00%,4.02%,0.340350


In [230]:
bondSymbpls = portfolio.get_bond_symbol_list()
bondSymbpls

['912797PW1',
 '912797QU4',
 '912797MG9',
 '912797QL4',
 '912797QT7',
 '912797QW0',
 '912797NA1']

In [270]:
portfolio.individualTransactions[portfolio.individualTransactions['Symbol']=='FZFXX']

,Run Date,Account,Account Number,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
1,2022-07-29,Individual,Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,0.22,NaT
2,2022-07-29,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.22,1.0,NaN,NaN,NaN,-0.22,NaT
15,2022-08-31,Individual,Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MAR...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,32.21,NaT
16,2022-08-31,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,32.21,1.0,NaN,NaN,NaN,-32.21,NaT
19,2022-09-30,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET F...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,34.02,1.0,NaN,NaN,NaN,-34.02,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,2025-05-30,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET FU...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,550.06,1.0,NaN,NaN,NaN,-550.06,NaT
556,2025-06-30,Individual,Z23390746,REINVESTMENT FIDELITY TREASURY MONEY MARKET FU...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,491.39,1.0,NaN,NaN,NaN,-491.39,NaT
559,2025-06-30,Individual,Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MARK...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,491.39,NaT
592,2025-07-31,Individual,Z23390746,DIVIDEND RECEIVED FIDELITY TREASURY MONEY MARK...,FZFXX,FIDELITY TREASURY MONEY MARKET FUND,Cash,0.00,NaN,NaN,NaN,NaN,111.12,NaT


In [239]:
symbols = portfolio.individualPosition['Symbol'].unique()
symbols

array(['FXAIX', '912797PW1', '912797QU4', 'FSKAX', '912797MG9',
       '912797QL4', '912797QT7', '912797QW0', 'FSPSX', '912797NA1',
       'FZFXX**', 'MSFT', 'AAPL', 'TSLA', 'MCD', 'BRKB', 'GOOGL', 'AXP',
       'JPM', 'KHC', 'SBUX', 'COKE', 'NKE', 'VZ', 'AMZN', 'NVDA', 'PG',
       'PEP', 'COST'], dtype=object)

In [240]:
symbols = portfolio.individualTransactions['Symbol'].unique()
symbols

array(['Transfer', 'FZFXX', 'AAPL', 'SBUX', 'JPM', 'AXP', 'GOOGL', 'AMZN',
       'NKE', '912797JK4', '912797LL9', '912797GN1', '912797GZ4',
       '912797HG5', 'TSLA', 'FXAIX', '912797HQ3', '912797HT7', 'MCD',
       '912797GB7', '912797HS9', '912797KV8', '912797JT5', '912797JU2',
       '912797KB2', '912797KC0', 'FSPSX', 'FSKAX', 'COKE', 'MSFT',
       '912797KL0', '912797LH8', '912797LE5', '912797KA4', '912797GW1',
       '912797HP5', '912797KT3', '912797LC9', '912797LR6', '912797LX3',
       '912797MB0', '912797LY1', 'BRKB', '912797LZ8', '912797LP0',
       '912796ZV4', 'KHC', '912797MX2', 'VZ', '912797KJ5', '912797NB9',
       '912797PJ0', '', '912797MM6', 'NVDA', '912797ND5', '912797NK9',
       'PEP', '912797PT8', '912797NC7', '912797NZ6', '912797NE3',
       '912797NN3', '912797LN5', '912797NV5', '912797QA8', '912797QB6',
       '912797QH3', '912797PW1', '912797PG6', '912797MG9', '912797QL4',
       '912797QU4', 'PG', 'COST', '912797QT7', '912797NA1', '912797QW0'],
      dtype=

# Attribute 1: get investment distribution

In [6]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

current_portfolio = Portfolio(transactions=transactions, position=position)

NameError: name 'transactions' is not defined

In [7]:
current_portfolio.show_investment_distribution()

   Class     Amount  Percent
0  stock  240727.72   40.12%
1   bill  345920.55   57.65%
2  other   13351.73    2.23%
3  total  600000.00  100.00%


# Attribute 2: get IRR for each stock

In [10]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

current_portfolio = Portfolio(transactions=transactions, position=position)

In [11]:
current_portfolio.show_stock_irr()

       Stock      irr
0   Transfer   11.57%
1       AAPL   19.95%
2       SBUX   14.71%
3        JPM   39.95%
4        AXP   35.91%
5      GOOGL   47.53%
6       AMZN   59.82%
7        NKE  -10.67%
8       TSLA  101.81%
9      FXAIX   30.93%
10       MCD   35.55%
11     FSKAX   34.87%
12     FSPSX   -5.18%
13      COKE   41.59%
14      MSFT  -12.08%
15      BRKB   30.43%
16     stock   23.08%


# Attribute 3: get irr of each portfolio

In [18]:
importlib.reload(portfolio)
Portfolio = portfolio.Portfolio

current_portfolio = Portfolio(transactions=transactions, position=position)

In [20]:
current_portfolio.pension_transactions
# current_portfolio.transactions[
#             current_portfolio.transactions["Account"] == "ERNST & YOUNG 401(K) 86964"
#         ].shape

,Run Date,Account,Action,Symbol,Description,Type,Quantity,Price ($),Commission ($),Fees ($),Accrued Interest ($),Amount ($),Settlement Date
103,2024-01-26,ERNST & YOUNG 401(K) 86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.070,NaN,NaN,NaN,NaN,242.30,NaT
104,2024-01-26,ERNST & YOUNG 401(K) 86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.716,NaN,NaN,NaN,NaN,103.85,NaT
114,2024-02-09,ERNST & YOUNG 401(K) 86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.041,NaN,NaN,NaN,NaN,242.30,NaT
115,2024-02-09,ERNST & YOUNG 401(K) 86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.542,NaN,NaN,NaN,NaN,103.85,NaT
122,2024-02-23,ERNST & YOUNG 401(K) 86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.493,NaN,NaN,NaN,NaN,103.85,NaT
123,2024-02-23,ERNST & YOUNG 401(K) 86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.027,NaN,NaN,NaN,NaN,242.30,NaT
134,2024-03-08,ERNST & YOUNG 401(K) 86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.408,NaN,NaN,NaN,NaN,103.85,NaT
135,2024-03-08,ERNST & YOUNG 401(K) 86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,1.020,NaN,NaN,NaN,NaN,242.30,NaT
138,2024-03-22,ERNST & YOUNG 401(K) 86964,Contributions,NaN,FID BLUE CHIP GR K6,NaN,3.336,NaN,NaN,NaN,NaN,103.85,NaT
139,2024-03-22,ERNST & YOUNG 401(K) 86964,Contributions,NaN,SP 500 INDEX PL CL E,NaN,0.997,NaN,NaN,NaN,NaN,242.30,NaT


In [267]:
current_portfolio.add_total_current_value_to_individual_position()
current_portfolio.set_merged_individual_position_transaction()
current_portfolio.add_time_diff_in_merged_individual_position_transaction()
unique_symbols = current_portfolio.merged_individual_position_transaction["Symbol"].unique()
stock_list = [
    element
    for element in unique_symbols
    if "FZFXX" not in element and not element[0].isdigit()
]
stock_list

Total current value has been added


['Transfer',
 'AAPL',
 'SBUX',
 'JPM',
 'AXP',
 'GOOGL',
 'AMZN',
 'NKE',
 'TSLA',
 'FXAIX',
 'MCD',
 'FSKAX',
 'FSPSX',
 'COKE',
 'MSFT',
 'BRKB',
 'Pending Activity']